In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import wandb
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
hf_datasets =  {'train': 'data/train-00000-of-00001-1359597a978bc4fa.parquet', 'valid': 'data/valid-00000-of-00001-70d52db3c749a935.parquet'}

In [3]:
df_train = pd.read_parquet("hf://datasets/zh-plus/tiny-imagenet/" + hf_datasets["train"])

df_train.shape


(100000, 2)

In [4]:
df_test = pd.read_parquet("hf://datasets/zh-plus/tiny-imagenet/" + hf_datasets["valid"])

df_test.shape

(10000, 2)

In [5]:
df_train.head()

,image,label
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0


In [15]:
from PIL import Image
from torchvision import transforms
import io

transform = transforms.ToTensor()

def load_image_data(series: pd.Series):

    arr = []
    for row in series:
        img = Image.open(io.BytesIO(row["bytes"]))
        if img.mode != "RGB":
            img = img.convert("RGB")
        tensor = transform(img)


        arr.append(tensor)
    
    return torch.stack(arr)




In [16]:
train_val_imgs = load_image_data(df_train['image'])


In [17]:
train_val_imgs.shape

torch.Size([100000, 3, 64, 64])

In [18]:
train_val_labels = torch.tensor(df_train["label"].tolist(), dtype=torch.long)


In [19]:
train_val_labels.shape

torch.Size([100000])

In [20]:
assert len(train_val_imgs) == len(train_val_labels), "Number of images and labels should be the same"

train_val_raw = list(zip(train_val_imgs, train_val_labels))

In [32]:
#hyperparameters
batch_size = 64
n_epochs = 35
lr = 0.001
factor = 0.5
patience = 4
early_stop_patience = 10

In [33]:
train_dataset, val_dataset = torch.utils.data.random_split(train_val_raw, [int(len(train_val_raw) * 0.8), int(len(train_val_raw) * 0.2)])

train_dataset = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [34]:
test_imgs = load_image_data(df_test['image'])
test_labels = torch.tensor(df_test["label"].tolist(), dtype=torch.long)

test_raw = list(zip(test_imgs, test_labels))

test_dataset = torch.utils.data.DataLoader(test_raw, batch_size=batch_size, shuffle=False)


In [37]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.sequential = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(64 * 6 * 6, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 200)
        )



        self.sequential.apply(self.__init_weights)
    
    def forward(self, x):

        x = self.sequential(x)

        return x
    
    def __init_weights(self,m):
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)

In [38]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = CNN().to(device)

In [39]:

optim = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=factor, patience=patience)


In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="bachelor-tinyimagenet",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "CNN",
    "dataset": "tiny-imagenet",
    "lr_scheduler": "ReduceLROnPlateau",
    "lr_scheduler_factor": factor,
    "lr_scheduler_patience": patience,
    "early_stop_patience": early_stop_patience,
    "epochs": n_epochs,
    "batch_size": batch_size
    }
)

In [40]:
es_counter = 0
min_val_loss = np.inf

for i in range(n_epochs):
    model.train()
    train_loss = 0
    train_acc = 0
    for x, y in tqdm(train_dataset):
        x = x.to(device)
        y = y.to(device)

        optim.zero_grad()
        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optim.step()

        train_loss += loss.item()
        train_acc += (y_pred.argmax(1) == y).sum().item() / y.size(0)

    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset)

    model.eval()
    val_loss = 0
    val_acc = 0
    for x, y in val_dataset:
        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)

        val_loss += loss.item()
        val_acc += (y_pred.argmax(1) == y).sum().item() / y.size(0)

    val_loss /= len(val_dataset)
    val_acc /= len(val_dataset)


    lr_scheduler.step(val_loss)

    print(f"Epoch {i + 1}/{n_epochs}")
    print(f"Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}")
    print(f"Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")

    wandb.log({
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })


    if val_loss < min_val_loss:
        min_val_loss = val_loss
        es_counter = 0
    else:
        es_counter += 1
        if es_counter >= early_stop_patience:
            print("Early stopping")
            break

 56%|█████▌    | 698/1250 [00:10<00:08, 68.86it/s]


KeyboardInterrupt: 

In [ ]:
wandb.finish()